In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
    
from tqdm import tqdm
import os
import platform
print(platform.python_version())
import sys
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
from collections import OrderedDict
from datetime import date
import zarr

# PHATHOM
from phathom import io

3.7.11


In [111]:
dpath = '/media/share5/mykim/CFC-2R'

In [112]:
cc_prefix = 'output_l1-7_all_converted.csv'
cp_prefix = 'count_cp_l1-7_all_converted.csv'
merged_fname = os.path.join(dpath, 'merged_all.csv')

In [113]:
# load csv
cc_fname = os.path.join(dpath, cc_prefix)
cp_fname= os.path.join(dpath, cp_prefix)
df_cc = pd.read_csv(cc_fname)
df_cp = pd.read_csv(cp_fname)

len(df_cc), len(df_cp)

(759, 688)

In [114]:
df_cc.head()

,id,counts,density,name,acronym,d7,d6,d5,d4,d3,d2,d1
0,2,241803,27.381427,Cerebrum,CH,Cerebrum,Cerebrum,Cerebrum,Cerebrum,Cerebrum,Cerebrum,Cerebrum
1,3,233147,32.879488,Cerebral cortex,CTX,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebrum
2,4,228995,33.646146,Cortical plate,CTXpl,Cortical plate,Cortical plate,Cortical plate,Cortical plate,Cortical plate,Cerebral cortex,Cerebrum
3,5,164065,41.568214,Isocortex,Isocortex,Isocortex,Isocortex,Isocortex,Isocortex,Cortical plate,Cerebral cortex,Cerebrum
4,6,553,17.775063,"Frontal pole, cerebral cortex",FRP,"Frontal pole, cerebral cortex","Frontal pole, cerebral cortex","Frontal pole, cerebral cortex",Isocortex,Cortical plate,Cerebral cortex,Cerebrum


In [115]:
df_cp.head()

,id,counts,density,name,acronym,d7,d6,d5,d4,d3,d2,d1
0,2,40374,4.571894,Cerebrum,CH,Cerebrum,Cerebrum,Cerebrum,Cerebrum,Cerebrum,Cerebrum,Cerebrum
1,3,38729,5.461746,Cerebral cortex,CTX,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebrum
2,4,37638,5.530137,Cortical plate,CTXpl,Cortical plate,Cortical plate,Cortical plate,Cortical plate,Cortical plate,Cerebral cortex,Cerebrum
3,5,28068,7.111429,Isocortex,Isocortex,Isocortex,Isocortex,Isocortex,Isocortex,Cortical plate,Cerebral cortex,Cerebrum
4,6,66,2.121436,"Frontal pole, cerebral cortex",FRP,"Frontal pole, cerebral cortex","Frontal pole, cerebral cortex","Frontal pole, cerebral cortex",Isocortex,Cortical plate,Cerebral cortex,Cerebrum


In [116]:
# rename co-positive columns
df_cp = df_cp.rename(columns={"counts": "cp_counts", "density": "cp_density"})
df_cp.head()

,id,cp_counts,cp_density,name,acronym,d7,d6,d5,d4,d3,d2,d1
0,2,40374,4.571894,Cerebrum,CH,Cerebrum,Cerebrum,Cerebrum,Cerebrum,Cerebrum,Cerebrum,Cerebrum
1,3,38729,5.461746,Cerebral cortex,CTX,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebrum
2,4,37638,5.530137,Cortical plate,CTXpl,Cortical plate,Cortical plate,Cortical plate,Cortical plate,Cortical plate,Cerebral cortex,Cerebrum
3,5,28068,7.111429,Isocortex,Isocortex,Isocortex,Isocortex,Isocortex,Isocortex,Cortical plate,Cerebral cortex,Cerebrum
4,6,66,2.121436,"Frontal pole, cerebral cortex",FRP,"Frontal pole, cerebral cortex","Frontal pole, cerebral cortex","Frontal pole, cerebral cortex",Isocortex,Cortical plate,Cerebral cortex,Cerebrum


#### Merge two DF

In [117]:
#cols_to_use = df_cp.columns.difference(df_cc.columns)
#df_cp[cols_to_use].head()

In [118]:
df_merged = pd.merge(df_cc, df_cp[['id', 'cp_counts', 'cp_density']], on=['id'], how="outer", indicator=True)
df_merged.head()

,id,counts,density,name,acronym,d7,d6,d5,d4,d3,d2,d1,cp_counts,cp_density,_merge
0,2,241803,27.381427,Cerebrum,CH,Cerebrum,Cerebrum,Cerebrum,Cerebrum,Cerebrum,Cerebrum,Cerebrum,40374.0,4.571894,both
1,3,233147,32.879488,Cerebral cortex,CTX,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebral cortex,Cerebrum,38729.0,5.461746,both
2,4,228995,33.646146,Cortical plate,CTXpl,Cortical plate,Cortical plate,Cortical plate,Cortical plate,Cortical plate,Cerebral cortex,Cerebrum,37638.0,5.530137,both
3,5,164065,41.568214,Isocortex,Isocortex,Isocortex,Isocortex,Isocortex,Isocortex,Cortical plate,Cerebral cortex,Cerebrum,28068.0,7.111429,both
4,6,553,17.775063,"Frontal pole, cerebral cortex",FRP,"Frontal pole, cerebral cortex","Frontal pole, cerebral cortex","Frontal pole, cerebral cortex",Isocortex,Cortical plate,Cerebral cortex,Cerebrum,66.0,2.121436,both


In [119]:
# save
df_merged.to_csv(merged_fname, sep=',', index=False)

In [120]:
tid = 6 # 457
a = df_cc[df_cc['id']==tid]
a

,id,counts,density,name,acronym,d7,d6,d5,d4,d3,d2,d1
4,6,553,17.775063,"Frontal pole, cerebral cortex",FRP,"Frontal pole, cerebral cortex","Frontal pole, cerebral cortex","Frontal pole, cerebral cortex",Isocortex,Cortical plate,Cerebral cortex,Cerebrum


In [121]:
b = df_cp[df_cp['id']==tid]
b

,id,cp_counts,cp_density,name,acronym,d7,d6,d5,d4,d3,d2,d1
4,6,66,2.121436,"Frontal pole, cerebral cortex",FRP,"Frontal pole, cerebral cortex","Frontal pole, cerebral cortex","Frontal pole, cerebral cortex",Isocortex,Cortical plate,Cerebral cortex,Cerebrum


In [122]:
c = df_merged[df_merged['id']==tid]
c

,id,counts,density,name,acronym,d7,d6,d5,d4,d3,d2,d1,cp_counts,cp_density,_merge
4,6,553,17.775063,"Frontal pole, cerebral cortex",FRP,"Frontal pole, cerebral cortex","Frontal pole, cerebral cortex","Frontal pole, cerebral cortex",Isocortex,Cortical plate,Cerebral cortex,Cerebrum,66.0,2.121436,both
